In [ ]:
import os, pytz, pandas as pd
from datetime import date, datetime, timedelta
from sqlalchemy import create_engine
import matplotlib.pyplot as plt, seaborn as sns

## parameters

In [ ]:
yeari, yearf = '2024', '2024'
weeki, weekf = '18', '31'

In [ ]:
di = datetime.strptime(f'{yeari}-{weeki}-1', "%Y-%W-%w").date()
df = datetime.strptime(f'{yearf}-{weekf}-1', "%Y-%W-%w").date() + timedelta(6)
ds = [di+timedelta(dt) for dt in range((df-di).days+1)]
daylist = ds
print(di, 'until', df)

In [ ]:
cdef = 'tl7_10m'# 'tl5_10m' 'tl6_10m' 'tl7_10m' 'tl8_10m' 'tl8_60m'
cdef_alt = '16m_10min'# tl5: 62 ... tl7: 16   tl8: 8

## database connection

In [ ]:
# database credentials
db_usr, db_pwd = os.getenv('DB_USR'), os.getenv('DB_PWD') # your database user name and password
# database login
host, port, db = 'nc-health-data-prod.cluster-ccsgl7rk4urn.eu-central-1.rds.amazonaws.com', 5432, 'master'

In [ ]:
# for queries with output
engine = create_engine('postgresql://'+db_usr+':'+db_pwd+'@'+host+':'+str(port)+'/'+db)
conn = engine.connect()

In [ ]:
conn.close()

## POI contacts

### shopping, outing, dining

In [ ]:
query = f"""
    with cn_poi as (
    	select *
    	from covid_network_sdkv6_{cdef} as cn
    	join euro2024_poi as poi on poi.tile_id = cn.tile_id
    	where "day" between '{str(di-timedelta(1))}' and '{str(df)}'
        and st_contains(poi.way_polygon, st_transform(cn.geopoint, 3857))
    )
    select amenity, shop, "day", stime, dids, st_x(st_transform(geopoint, 4326)) as lon, st_y(st_transform(geopoint, 4326)) as lat, tl7--, way_polygon, geopoint
    from cn_poi
"""
data_1 = pd.DataFrame(pd.read_sql_query(query, conn))

In [ ]:
data_1['stime'] = data_1.stime.apply(lambda x: x.astimezone(pytz.timezone('Europe/Berlin')))
data_1['day'] = data_1.stime.apply(lambda x: x.date())
data_1 = data_1[(data_1.day >= di) & (data_1.day <= df)]

In [ ]:
data_sv = data_1.copy(deep=True)

In [ ]:
data_1 = data_sv.copy(deep=True)

In [ ]:
data_amenity = data_1[[col for col in data_1.columns if col!='shop']]
data_amenity = data_amenity[~data_amenity.amenity.isna()]
data_amenity = data_amenity[data_amenity.amenity.isin(['bar','restaurant','biergarten','pub','cafe'])]
data_amenity = data_amenity.rename(columns={'amenity':'venue'})
data_shop = data_1[[col for col in data_1.columns if col!='amenity']]
data_shop = data_shop[~data_shop.shop.isna()]
data_shop = data_shop[data_shop.shop.isin(['kiosk','supermarket','beverages'])]
data_shop = data_shop.rename(columns={'shop':'venue'})
data_1 = pd.concat([data_amenity, data_shop])
data_1

### fan zones

17707818
178050313
-7360802
143672616
-222512 146613649
165636449
172420880 (172341584)
4218882
-3538248
4979228
-3995902
-6404164
10053878
3933618
3990007
507962489
3996756
231448110, 4567556, 503322108, 29065531, 147841549, 532727925 (line)
-1769862
417328807, 278980053, 417328805 (point)
499593498
24240304

In [ ]:
query = f"""
    with osm_polygons as (
    	select osm_id, name, way
    	from planet_osm_polygon
    	where osm_id in (17707818,178050313,-7360802,143672616,-222512,146613649,165636449,172420880,172341584,4218882,
    					 -3538248,4979228,-3995902,-6404164,10053878,3933618,3990007,507962489,3996756,-1769862,499593498,24240304)
    )
    , osm_lines as (
    	select osm_id, name, way
    	from planet_osm_line
    	where osm_id in (231448110, 4567556, 503322108, 29065531, 147841549, 532727925)
    )
    , osm_points as (
    	select osm_id, name, way
    	from planet_osm_point
    	where osm_id in (417328807, 278980053, 417328805)
    )
    , osm as (
    	select *
    	from osm_polygons
    	union all
    	select *
    	from osm_lines
    	union all
    	select *
    	from osm_points
    )
    select "day", stime, dids, st_x(st_transform(geopoint, 4326)) as lon, st_y(st_transform(geopoint, 4326)) as lat, tl7, osm.osm_id, osm.name--, cn.geopoint, osm.osm_id, osm.way
    from covid_network_sdkv6_{cdef} as cn, osm
    where "day" between '{str(di-timedelta(1))}' and '{str(df)}'
    and st_distance(st_transform(cn.geopoint, 3857), osm.way) <= 100
"""
data_2 = pd.DataFrame(pd.read_sql_query(query, conn))
data_2

In [ ]:
data_2['stime'] = data_2.stime.apply(lambda x: x.astimezone(pytz.timezone('Europe/Berlin')))
data_2['day'] = data_2.stime.apply(lambda x: x.date())
data_2 = data_2[(data_2.day >= di) & (data_2.day <= df)]

In [ ]:
data_sv = data_2.copy(deep=True)

In [ ]:
data_2 = data_sv.copy(deep=True)

In [ ]:
city2zone = {
    'Berlin':['Platz der Republik','Platz des 18. März','Straße des 17. Juni'],
    'Hamburg':['Heiligengeistfeld'],
    'Gelsenkirchen':['Nordsternpark','Nordsternplatz'],
    'Dortmund':['Friedensplatz','Westfalenpark'],
    'Düsseldorf':['Burgplatz','Gustav-Gründgens-Platz','Rheinpark','Rheinwerft'],
    'Köln':['Aachener Weiher','Heumarkt'],
    'Leipzig':['Dr.-Otto-Koch-Denkmal','Gellert-Denkmal','Moritzbastei','Robert-Koch-Park','Schiller-Denkmal','Tiefgarage Augustusplatz'],
    'Stuttgart':['Karlsplatz','Marktplatz','Schillerplatz','Schlossplatz'],
    'München':['Olympiapark / Olympiagelände'],
    'Frankfurt am Main':['Nizza','Untermainkai'],
}
zone2city = {}
for city, zones in city2zone.items():
    for zone in zones:
        zone2city[zone] = city
data_2['city'] = data_2.name.map(zone2city)

In [ ]:
by_polygon = data_2.groupby(['day','city']).dids.apply(len).reset_index()
c1 = pd.DataFrame(set(by_polygon.city), columns=['city'])
c2 = pd.DataFrame([d.date() for d in pd.date_range(di, df)], columns=['day'])
by_polygon = c1.merge(c2, how='cross').merge(by_polygon, on=['city','day'], how='left')
by_polygon

In [ ]:
# Draw a heatmap with the numeric values in each cell
f, ax = plt.subplots(figsize=(12.5, 1.5))

pivoted = by_polygon.pivot(index="city", columns="day", values="dids")  # Reshape
g = sns.heatmap(pivoted.fillna(0), cmap="coolwarm", cbar=False, robust=True)#, **kwargs)

g.set(yticks=[x+.5 for x in range(len(pivoted.index))])
g.set_yticklabels(pivoted.index, rotation=0)

# Set yticks for all subplots
g.set(xticks=[x+.5 for x in range(len(set(data_2.day)))])
g.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in sorted(set(data_2.day))], rotation=0)
g.set_ylabel('fanzone')

plt.savefig(f'plots/fig3_{cdef_alt}/07_contacts_fanzone.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig3_{cdef_alt}/07_contacts_fanzone.pdf', bbox_inches='tight')
plt.show()

In [ ]:
data_2 = data_2.drop(columns=['osm_id','name'])
data_2['venue'] = 'fanzone'
data_2

### all POI together

In [ ]:
data = pd.concat([data_1, data_2])
data

In [ ]:
data = data.explode('dids').reset_index(drop=True)
data = data.drop_duplicates()
data = data.merge(data.drop(columns=['lon','lat','city']), on=['day','stime','tl7','venue',])#,'tl7','inside_building','lon','lat'])
data = data[data.dids_x != data.dids_y]
pairs = []
for did1, did2 in zip(data.dids_x, data.dids_y):
    pair = f'{did1}_{did2}' if did1 < did2 else f'{did2}_{did1}'
    #print(did1, did2, pair)
    pairs.append(pair)
data.loc[:,'pair'] = pairs
data = data.drop(columns=['dids_x','dids_y','tl7'])
data = data.drop_duplicates()
dmin = data.day.min()
#data['tt'] = data.day.apply(lambda d: (d-dmin).days)*24 + data.stime.dt.hour
data['tt'] = data.day.apply(lambda d: (d-dmin).days)*24*6 + data.stime.dt.hour*6 + (data.stime.dt.hour//10)
print(data.tt.max(), ((data.day.max()-dmin).days+1)*24, ((data.day.max()-dmin).days+1)*720)
data

In [ ]:
data.to_csv('output/07_poi_contacts.csv', index=False)

In [ ]:
data = pd.read_csv('output/07_poi_contacts.csv')
data['day'] = [d.date() for d in pd.to_datetime(data.day)]
data['stime'] = pd.to_datetime(data.stime)
data['hour'] = data.stime.dt.hour

In [ ]:
type2venue = {
    'shopping':['supermarket','beverages','kiosk'],
    'outing':['pub','bar','biergarten'],
    'dining':['restaurant','cafe'],
    'fanzone':['fanzone'],
}
venue2type = {}
for cl, venues in type2venue.items():
    for venue in venues:
        venue2type[venue] = cl
data['venue_type'] = data.venue.map(venue2type)
data

In [ ]:
data.groupby('venue').pair.apply(len)

In [ ]:
data.groupby('venue_type').pair.apply(len)

In [ ]:
for_heatmap = data.groupby(['venue_type','day','hour']).pair.apply(lambda x: len(set(x))).reset_index()
c1 = pd.DataFrame(set(for_heatmap.venue_type), columns=['venue_type'])
c2 = pd.DataFrame([d.date() for d in pd.date_range(di, df)], columns=['day'])
c3 = pd.DataFrame(list(range(24)), columns=['hour'])
for_heatmap = c1.merge(c2, how='cross').merge(c3, how='cross').merge(for_heatmap, on=['venue_type','day','hour'], how='left')
for_heatmap

In [ ]:
sns.set_theme(style="ticks")

# Function to plot a heatmap inside each facet
def heatmap_func(data, **kwargs):
    pivoted = data.pivot(index="hour", columns="day", values="pair")  # Reshape
    sns.heatmap(pivoted.loc[::-1], cmap="coolwarm", cbar=False, robust=True, **kwargs)

# Create a FacetGrid, grouping by 'category'
g = sns.FacetGrid(for_heatmap.fillna(0), row="venue_type", margin_titles=True, height=3, aspect=3.,
                  row_order=['dining','outing','fanzone','shopping'])

# Map the custom heatmap function to each facet
g.map_dataframe(heatmap_func)

# Set yticks for all subplots
g.set(yticks=[24-x-.5 for x in range(0,24,2)], xticks=[x+.5 for x in range(len(set(data.day)))])
g.set_yticklabels(list(range(0,24,2)), rotation=0)
g.set_xticklabels([str(d.month).zfill(2)+'/'+str(d.day).zfill(2) if d.weekday()==6 else '' for d in sorted(set(data.day))], rotation=0)

for ax in g.axes.flat:
    #ax.set_title(ax.get_title().split('=')[1][1:])
    ax.tick_params(axis="x", labelbottom=True)

# Modify margin titles
g.set_titles(row_template="{row_name}")#, col_template="{col_name}", size=14, fontweight='bold')

g.tight_layout()
plt.savefig(f'plots/fig3_{cdef_alt}/07_contacts_poi.jpg', bbox_inches='tight', dpi=300)
plt.savefig(f'plots/fig3_{cdef_alt}/07_contacts_poi.pdf', bbox_inches='tight')
plt.show()

In [ ]:
for_heatmap['wd'] = for_heatmap.day.apply(lambda x: x.weekday())
for_heatmap

In [ ]:
baseline = for_heatmap[for_heatmap.day>=date(2024,6,3)].groupby(['venue','wd','hour']).pair.mean().reset_index().rename(columns={'pair':'baseline'})
baseline

In [ ]:
to_baseline = for_heatmap[for_heatmap.day>=date(2024,6,3)].merge(baseline, on=['venue','wd','hour'])
to_baseline['to_baseline'] = to_baseline.pair / to_baseline.baseline
to_baseline

In [ ]:
# Function to plot a heatmap inside each facet
def heatmap_func(data, **kwargs):
    pivoted = data.pivot(index="hour", columns="day", values="to_baseline")  # Reshape
    sns.heatmap(pivoted.loc[::-1], cmap="coolwarm", cbar=True, center=1, robust=True, **kwargs)

# Create a FacetGrid, grouping by 'category'
g = sns.FacetGrid(to_baseline, row="venue", margin_titles=True, height=3, aspect=4)

# Map the custom heatmap function to each facet
g.map_dataframe(heatmap_func)

plt.show()